<a href="https://colab.research.google.com/github/nico-lina/challenge_aibot/blob/main/agentai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#librerie

In [ ]:
!pip install datasets chromadb gradio mistralai transformers sentence_transformers langchain_community

ERROR: Operation cancelled by user


#Attach to Drive

In [ ]:
import os
import sys
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/AgentAI_Semantic_Similarity/data')
sys.path.append('/content/drive/MyDrive/AgentAI_Semantic_Similarity/backend')
sys.path.append('/content/drive/MyDrive/AgentAI_Semantic_Similarity/frontend')
sys.path.append('/content/drive/MyDrive/AgentAI_Semantic_Similarity/model')
sys.path.append('/content/drive/MyDrive/AgentAI_Semantic_Similarity/VDB')
os.chdir('/content/drive/MyDrive/AgentAI_Semantic_Similarity/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Preparation

In [ ]:
import requests
import json
import requests
import os
import pandas as pd
from google.colab import userdata
from collections import defaultdict

def download_scryfall_bulk_data():
    """
    Recupera il 'download_uri' dall'API Scryfall per i dati di massa delle carte predefinite
    e scarica il file JSON associato nella directory specificata.

    """
    api_url = "https://api.scryfall.com/bulk-data/oracle_cards"
    full_output_path = userdata.get('card_path')
    try:
        response_api = requests.get(api_url)
        response_api.raise_for_status() # Solleva un'eccezione se ci sono codici d'errore
        data = response_api.json()

        download_uri = data.get("download_uri")
        if not download_uri:
            print("La chiave 'download_uri' non è stata trovata nell'oggetto JSON.")
            return
        response_download = requests.get(download_uri, stream=True)
        response_download.raise_for_status()
        with open(full_output_path, 'wb') as f:
            for chunk in response_download.iter_content(chunk_size=8192):
                f.write(chunk)
    except requests.exceptions.RequestException as e:
        print(f"Errore durante la richiesta API o il download: {e}")
    except ValueError as e:
        print(f"Errore durante il parsing del JSON: {e}")
    except Exception as e:
        print(f"Si è verificato un errore inatteso: {e}")

def select_columns():
  """
  Seleziona le colonne che dovranno essere filtrate e salvate in un nuovo file JSON.
  Successivamente subiranno una fase di data preparetion
  """
  file_path=userdata.get('card_path')
  with open(file_path, 'r', encoding = 'utf-8') as f:
    data = json.load(f)
  selected_columns = ['name','cmc', 'type_line', 'oracle_text', 'power', 'toughness', 'color_identity', 'keywords','legalities','image_uris']
  df = pd.DataFrame(data)[selected_columns]

  df.to_json(userdata.get('filtered_card'), orient = 'records')


def data_preparation():
  with open(userdata.get('filtered_card'), 'r', encoding = 'utf-8') as f:
    data = json.load(f)
  name_card_set = set()
  sub_type_set = set()
  type_set = set()
  keyword_set = set()
  type_set.add("")
  sub_type_set.add("")
  correct_db = []
  separator = "—"
  legalita_mapping = {
      "standard": 0,      # 2^0 = 1
      "future": 1,        # 2^1 = 2
      "historic": 2,      # 2^2 = 4
      "timeless": 3,      # 2^3 = 8
      "gladiator": 4,     # 2^4 = 16
      "pioneer": 5,       # 2^5 = 32
      "explorer": 6,      # 2^6 = 64
      "modern": 7,        # 2^7 = 128
      "legacy": 8,        # 2^8 = 256
      "pauper": 9,        # 2^9 = 512
      "vintage": 10,       # 2^10 = 1024
      "penny": 11,        # 2^11 = 2048
      "commander": 12,     # 2^12 = 4096
      "oathbreaker": 13,   # 2^13 = 8192
      "standardbrawl": 14, # 2^14 = 16384
      "brawl": 15,        # 2^15 = 32768
      "alchemy": 16,      # 2^16 = 65536
      "paupercommander": 17,# 2^17 = 131072
      "duel": 18,         # 2^18 = 262144
      "oldschool": 19,     # 2^19 = 524288
      "premodern": 20,     # 2^20 = 1048576
      "predh": 21          # 2^21 = 2097152
  }

  keyword_counts = defaultdict(int)
  dont_swap=['Awaken','Partner','Channel','Cipher','Conspire','Cycling','Dash','Dredge','Embalm','Encore','Entwine','Escape','Eternalize','Evoke ','Flashback','Forecast','Foretell','Fuse','Grandeur','Hideaway','Jump-start','Overload','Plot','Rebound','Recover','Reinforce','Retrace','Unearth','Venture into the Dungeon' ]
  for card in data:
      type_line = card.get('type_line', '')
      type_c = ''
      sub_type = ''
      legendary = 0
      bitmask = 0
      # Seleziono solo le carte valide
      if "Token" not in type_line and "Emblem" not in type_line and "Summon" not in type_line and card.get('name', '') not in name_card_set and '//' not in card.get('name', ''):
          name_card_set.add(card.get('name', ''))

          if separator in type_line:
              segments = type_line.split(separator, 1)
              type_c = segments[0].strip()  # Prelevo il tipo
              sub_type = segments[1].strip()  # Prelevo il sottotipo
              # Salvo tutti i tipi e sottotipi e tutte le loro combinazioni possibili
              sub_type_set.add(sub_type)
          else:
              type_c= type_line
              sub_type = 'tappo'

          if "Legendary" in type_c:  # Gestisco il leggendario
              legendary = 1
              type_c = type_c.replace("Legendary", "").strip()
          type_set.add(type_c)
          if card.get('color_identity', '') == []:  # Sistemo il campo per il colorless
              card['color_identity'] = ['CL']

          if card.get('keywords'):  # Salvo tutte le keyword esistenti
              for keyword in card.get('keywords'):
                  keyword_set.add(keyword)


          row = card.get('oracle_text', '').split('\n')
          # Se esistono più righe nella carta e corrisponde ad uno dei seguenti tipi e
          #non sono presenti le parole chiave in dont_swap sposto la prima sezione infondo al testo
          if (len(row) > 1 and
             ("Creature" in type_c or "Sorcery" in type_c or "Instant" in type_c) and
             all(word not in card['oracle_text'] for word in dont_swap)):
              card['oracle_text'] = '\n'.join(row[1:] + [row[0]])

          if "Enchantment" in type_c and "God" in sub_type:
              card['oracle_text'] = '\n'.join(row[1:] + [row[0]])
          for format, leg in card.get("legalities").items():
            if leg == "legal" or leg == "restricted":
              bitmask |= (1 << legalita_mapping[format])
          for uri_type,uri in card.get("image_uris").items():
            if uri_type == "border_crop":
              card["image_uris"]=uri
          card['legalities']=bitmask
          # Aggiungi i nuovi campi "Legendary", "Type", "SubType" alla carta
          card['Legendary'] = legendary
          card['Type'] = type_c
          card['SubType'] = sub_type

          correct_db.append(card)

  with open('/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/filtered_cards.json', 'w', encoding = 'utf-8') as f:
      json.dump(correct_db, f, indent= 4)
  with open('/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/name_cards.json', 'w', encoding = 'utf-8') as f:
      json.dump(list(name_card_set), f, indent= 4)
  with open('/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/type.json', 'w', encoding = 'utf-8') as f:
      json.dump(list(type_set), f, indent= 4)
  with open('/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/sub_type.json', 'w', encoding = 'utf-8') as f:
      json.dump(list(sub_type_set), f, indent= 4)
  with open('/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/keyword.json', 'w', encoding = 'utf-8') as f:
      json.dump(list(keyword_set), f, indent= 4)


download_scryfall_bulk_data()
select_columns()
data_preparation()




#Create VectorDatabase

In [ ]:
'''from transformers import AutoModel, AutoTokenizer

# 1. Load the model and tokenizer
model_name = 'sentence-transformers/all-mpnet-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Define the output path
output_path = '/content/drive/MyDrive/AgentAI_Semantic_Similarity/model_output'

# 3. Save the model and tokenizer
#tokenizer.save_pretrained(output_path)
#model.save_pretrained(output_path)

print(f"Model '{model_name}' saved to '{output_path}'")'''
'''import shutil
import os

# Define the path to your ChromaDB persistent client data
chroma_db_path = "/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/VDB_mod_2"

# Check if the directory exists before attempting to delete it
if os.path.exists(chroma_db_path):
    try:
        shutil.rmtree(chroma_db_path)
        print(f"Directory '{chroma_db_path}' and its contents deleted successfully.")
    except Exception as e:
        print(f"Error deleting directory '{chroma_db_path}': {e}")
else:
    print(f"Directory '{chroma_db_path}' does not exist. No need to delete.")
'''

'import shutil\nimport os\n\n# Define the path to your ChromaDB persistent client data\nchroma_db_path = "/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/VDB_mod_2"\n\n# Check if the directory exists before attempting to delete it\nif os.path.exists(chroma_db_path):\n    try:\n        shutil.rmtree(chroma_db_path)\n        print(f"Directory \'{chroma_db_path}\' and its contents deleted successfully.")\n    except Exception as e:\n        print(f"Error deleting directory \'{chroma_db_path}\': {e}")\nelse:\n    print(f"Directory \'{chroma_db_path}\' does not exist. No need to delete.")\n'

In [ ]:
import json
import chromadb
import torch
from sentence_transformers import SentenceTransformer
from chromadb import EmbeddingFunction, Documents, Embeddings

# Percorso al modello Sentence-BERT fine-tunato
model_path = "/content/drive/MyDrive/AgentAI_Semantic_Similarity/model_output"
#model_path = "/content/drive/MyDrive/AgentAI_Semantic_Similarity/model_mod_1"
# Verifica se la GPU è disponibile
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilizzando il dispositivo: {device}")

# Definizione della funzione di embedding con SentenceTransformer
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model_path: str, device: str):
        self.model = SentenceTransformer(model_path, device=device)

    def __call__(self, input: Documents) -> Embeddings:
        embeddings = self.model.encode(input, convert_to_numpy=True, show_progress_bar=False)
        return embeddings.tolist()

# Crea l'istanza della funzione di embedding
embedding_function = MyEmbeddingFunction(model_path, device)

# Inizializza ChromaDB
client = chromadb.PersistentClient(path="/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/VDB_mod_2")


################################################################################################
collection_name = "magic_vdb_mod_2"
try:
    client.delete_collection(name=collection_name)
    print(f"Collezione '{collection_name}' cancellata con successo (se esisteva).")
except Exception as e:
    print(f"Impossibile cancellare la collezione '{collection_name}' (potrebbe non esistere): {e}")
###############################################################################################
# Crea la nuova collezione con i parametri HNSW aggiornati
collection = client.get_or_create_collection(
    name="magic_vdb_mod_2",
    embedding_function=embedding_function,
    configuration={
        "hnsw":{"space": "cosine",
                "ef_construction": 4000,
                "ef_search": 500,
                 "max_neighbors": 32}
    }
)
print("Collezione 'magic_vdb' creata/ottenuta con i parametri HNSW aggiornati.")


# Carica le carte filtrate
with open('/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/filtered_cards.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

selected_columns = ['name', 'cmc', 'type_line', 'power', 'toughness', 'color_identity', 'keywords', 'legalities', 'image_uris', 'Legendary', 'Type', 'SubType']

# Funzione per elaborare un batch di carte
def process_batch(batch, start_id):
    documents = []
    metadatas = []
    ids = []
    for i, obj in enumerate(batch):
        oracle_text = obj.get("oracle_text", "")
        metadata_entry = {}
        for key in selected_columns:
            value = obj.get(key)
            if value is None:
                metadata_entry[key] = ""
            elif isinstance(value, list):
                metadata_entry[key] = ", ".join(map(str, value))
            elif isinstance(value, dict):
                metadata_entry[key] = json.dumps(value)
            else:
                metadata_entry[key] = value

        card_id = f"card_{start_id + i}"
        metadata_entry["id"] = card_id

        documents.append(oracle_text)
        metadatas.append(metadata_entry)
        ids.append(card_id)

    # Aggiunta delle carte a ChromaDB
    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )

# Elaborazione delle carte in batch
batch_size = 400
current_id = 0
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    process_batch(batch, current_id)
    current_id += len(batch)
    print(f"Elaborato e aggiunto il batch {i // batch_size + 1}")
    torch.cuda.empty_cache()

print(f"Elaborato e inserito un totale di {len(data)} carte in Chroma")

Utilizzando il dispositivo: cuda
Collezione 'magic_vdb_mod_2' cancellata con successo (se esisteva).
Collezione 'magic_vdb' creata/ottenuta con i parametri HNSW aggiornati.
Elaborato e aggiunto il batch 1
Elaborato e aggiunto il batch 2
Elaborato e aggiunto il batch 3
Elaborato e aggiunto il batch 4
Elaborato e aggiunto il batch 5
Elaborato e aggiunto il batch 6
Elaborato e aggiunto il batch 7
Elaborato e aggiunto il batch 8
Elaborato e aggiunto il batch 9
Elaborato e aggiunto il batch 10
Elaborato e aggiunto il batch 11
Elaborato e aggiunto il batch 12
Elaborato e aggiunto il batch 13
Elaborato e aggiunto il batch 14
Elaborato e aggiunto il batch 15
Elaborato e aggiunto il batch 16
Elaborato e aggiunto il batch 17
Elaborato e aggiunto il batch 18
Elaborato e aggiunto il batch 19
Elaborato e aggiunto il batch 20
Elaborato e aggiunto il batch 21
Elaborato e aggiunto il batch 22
Elaborato e aggiunto il batch 23
Elaborato e aggiunto il batch 24
Elaborato e aggiunto il batch 25
Elaborato e

In [ ]:
import json
import torch
from sentence_transformers import SentenceTransformer

# Percorso al modello Sentence-BERT fine-tunato
model_path = "/content/drive/MyDrive/AgentAI_Semantic_Similarity/model_output"
# Percorso al file JSON di input
input_json_path = '/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/filtered_cards.json'
# Percorso al file JSON di output con gli embedding
output_json_path = '/content/drive/MyDrive/AgentAI_Semantic_Similarity/data/filtered_cards_with_embeddings.json'

# Verifica se la GPU è disponibile
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilizzando il dispositivo: {device}")

# Carica il modello SentenceTransformer
try:
    model = SentenceTransformer(model_path, device=device)
    print(f"Modello caricato da '{model_path}' sul dispositivo '{device}'.")
except Exception as e:
    print(f"Errore durante il caricamento del modello: {e}")
    print("Assicurati che il percorso del modello sia corretto e che il modello sia stato salvato correttamente.")
    exit() # Esci se il modello non può essere caricato

# Carica le carte filtrate dal file JSON
try:
    with open(input_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"Caricate {len(data)} carte da '{input_json_path}'.")
except FileNotFoundError:
    print(f"Errore: Il file '{input_json_path}' non è stato trovato.")
    exit()
except json.JSONDecodeError:
    print(f"Errore: Impossibile decodificare il file JSON '{input_json_path}'. Controlla la sua formattazione.")
    exit()

# Contatore per le carte elaborate
processed_cards_count = 0

# Itera su ogni carta per generare e aggiungere l'embedding
print("Generazione degli embedding e aggiunta al file JSON...")
for i, card in enumerate(data):
    oracle_text = card.get("oracle_text", "")

    # Genera l'embedding per il testo oracle
    # L'embedding viene convertito in una lista Python per essere salvato nel JSON
    try:
        embedding = model.encode(oracle_text, convert_to_numpy=True, show_progress_bar=False)
        card['embedding'] = embedding.tolist()
        processed_cards_count += 1
        if (i + 1) % 100 == 0: # Stampa un aggiornamento ogni 100 carte
            print(f"Elaborate {i + 1} carte...")
    except Exception as e:
        print(f"Errore durante la generazione dell'embedding per la carta {card.get('name', 'Sconosciuta')}: {e}")
        card['embedding'] = None # Imposta l'embedding a None in caso di errore

    # Libera la memoria GPU dopo ogni embedding (buona pratica per batch più piccoli o iterazioni)
    torch.cuda.empty_cache()

print(f"Generati e aggiunti gli embedding per {processed_cards_count} carte.")

# Salva i dati aggiornati in un nuovo file JSON
try:
    with open(input_json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Dati con gli embedding salvati con successo in '{input_json_path}'.")
except Exception as e:
    print(f"Errore durante il salvataggio del file JSON: {e}")



#GUI

In [ ]:
import importlib
import UI_gradio

prova=UI_gradio.start_UI().launch(share=True, debug = True)

/content/drive/MyDrive/AgentAI_Semantic_Similarity/frontend/agentetool.py:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/content/drive/MyDrive/AgentAI_Semantic_Similarity/frontend/agentetool.py:93: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/content/drive/MyDrive/AgentAI_Semantic_Similarity/frontend/agentetool.py:96: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible 

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://eb1868db6fc611b809.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/content/drive/MyDrive/AgentAI_Semantic_Similarity/frontend/agentetool.py:115: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent_executor.run(user_input)




> Entering new AgentExecutor chain...
Thought: The user wants a card that, when it enters the battlefield, allows them to draw two cards. This is a common ability in Magic: The Gathering, often seen on creatures, artifacts, or enchantments.
Action: No need to use the query tool as the request is clear and straightforward.
Final Answer:
When this permanent enters the battlefield, draw two cards.

> Finished chain.
-------FLAG = 0------
[]


> Entering new AgentExecutor chain...
Thought: The user seems to be asking to search for a card that matches the previously described concept. I will use the query tool to find similar cards.

Action: query
Action Input: "When this permanent enters the battlefield, draw two cards."
Utilizzando il dispositivo: cuda

Observation: [{'image_url': 'https://cards.scryfall.io/border_crop/front/8/f/8fbb37d7-7053-4b1f-b899-91695f88f7e0.jpg?1674137796', 'distance': 0.3795604109764099}, {'image_url': 'https://cards.scryfall.io/border_crop/front/e/3/e3151960-c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Agent AI

In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import RAG_Connection as conn

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")

user_input = input("")

llm = ChatOpenAI(
    model="mistral-medium",
    base_url="https://api.mistral.ai/v1",
    api_key=api_key,
    temperature=0,
)

def query_empty_where(oracle_text : str):
    """Funzione che esegue una query su un database VDB"""
    #Connessione al VDB
    collection = conn.connect_to_vdb()
    result=collection.query(
        query_texts=[oracle_text], #testo da inserire e verrà embeddato a runtime
        n_results=30#numero d carte che si vogliono
    )
    return result

def translate(text: str) -> str:
    response = llm.invoke("Translate the following text to English: " + text)
    return response.content

def query(text: str):
  return query_empty_where(translate(text))

tools = [
    Tool(
        name="query",
        func=query,
        description="Use this tool to retrive the cards with the chosen text"
    )
]


memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)

result = agent.run(user_input)
print(result)

KeyboardInterrupt: Interrupted by user